In [26]:
import pandas as pd 
import numpy as np 

Dropping data now after Excel cleanong, we imputted as much data as possible , now im going to drop data with values nan and "No Value" from each column 

In [27]:
#xls = pd.ExcelFile("Excel Clean.xlsx")
#print(xls.sheet_names)  # Lists all sheet names

df = pd.read_excel("Excel Clean.xlsx", sheet_name="Cleaned_Cafe_Data")



In [28]:
df.head()



,Transaction ID,Items,Quantity,Price Per Unit,Grand total,Payment Method,Location,Transaction Date,Days_of_week,Location_was_unavailable,Unnamed: 10,Unnamed: 11,Product,Price
0,TXN_1000555,Tea,1,1.5,1.5,Credit Card,In-store,2023-10-19,1900-01-05,False,NaN,NaN,Cake,3.0
1,TXN_1001832,Salad,2,5,10,Cash,Takeaway,2023-10-19,1900-01-05,False,NaN,NaN,Coffee,2.0
2,TXN_1002457,Cookie,5,1,5,Digital Wallet,Takeaway,2023-09-29,1900-01-06,False,NaN,NaN,Cookie,1.0
3,TXN_1003246,Juice,2,3,6,NaN,NaN,2023-02-15,1900-01-04,True,NaN,NaN,Juice,3.0
4,TXN_1004184,Smoothie,1,4,4,Credit Card,In-store,2023-05-18,1900-01-05,False,NaN,NaN,Salad,5.0


In [29]:
(df.isna().sum() + (df == "No Value").sum())


Transaction ID                  0
Items                         512
Quantity                       18
Price Per Unit                 38
Grand total                    20
Payment Method               3178
Location                     3961
Transaction Date                0
Days_of_week                    0
Location_was_unavailable        0
Unnamed: 10                 10000
Unnamed: 11                 10000
Product                      9992
Price                        9992
dtype: int64

drop na and no value cells from items,quantity, price per unit and grand total

In [30]:
cols_to_clean = ['Items', 'Quantity', 'Price Per Unit', 'Grand total']


Check if this row contains 'no value' (even if that 'no value' was typed differently), but don’t mess with the actual data types while doing it.
df[cols_to_clean]
→ Select just the important columns you're cleaning (like Items, Quantity, etc.).

.apply(..., axis=1)
→ Go through each row of those columns.

lambda row: ...
→ Temporarily convert each cell in the row to a string, clean it (strip spaces, lowercase), and check if it equals "no value".

.any(axis=1)
→ Now, for each row, ask: "Was any cell equal to 'no value'?"

If yes → this row is bad, we want to remove it.

If no → this row is good, we want to keep it.

~ (tilde)
→ Flip the logic! Now we're keeping rows that do NOT contain "no value".

df[...]
→ We're using that result to filter the DataFrame and assign it back to df.



In [31]:
# Step 1: Drop rows with "No Value" (case-insensitive string match)
#df = df[~df[cols_to_clean].apply( # ~ Keep only the rows that do NOT have "no value" in them.  .apply: This applies a function to each row (axis=1 = row-wise).
#    lambda row: row.astype(str).str.strip().str.lower().eq("no value"), axis=1 # This defines what to do for each row of those selected columns.
#).any(axis=1)]


In [32]:
# Step 2: Drop NaNs in those columns
#df.dropna(subset=cols_to_clean, inplace=True)

In [33]:
# Step 3: Drop rows with 0 in Quantity, Price Per Unit, or Grand total
#df = df[~((df['Quantity'] == 0) | (df['Price Per Unit'] == 0) | (df['Grand total'] == 0))]

In [34]:
# Step 4: Reset index
# df.reset_index(drop=True, inplace=True)

ANOTHER APPROACH


In [35]:
'''cols_to_check = ["Items", "Quantity", "Price Per Unit", "Grand total"]

# Check how many 'bad' values are in each row for those 4 columns
bad_counts = df[cols_to_check].apply(
    lambda row: sum((pd.isna(row)) | (row.astype(str).str.strip().str.lower() == "no value")),
    axis=1
)

# Count how many rows had 3 or more bad values
print("Rows with 2 or more bad values in important columns:", (bad_counts >= 2).sum())

df = df[bad_counts < 2]  # keep rows with 0, 1, or bad values
'''

'cols_to_check = ["Items", "Quantity", "Price Per Unit", "Grand total"]\n\n# Check how many \'bad\' values are in each row for those 4 columns\nbad_counts = df[cols_to_check].apply(\n    lambda row: sum((pd.isna(row)) | (row.astype(str).str.strip().str.lower() == "no value")),\n    axis=1\n)\n\n# Count how many rows had 3 or more bad values\nprint("Rows with 2 or more bad values in important columns:", (bad_counts >= 2).sum())\n\ndf = df[bad_counts < 2]  # keep rows with 0, 1, or bad values\n'

An idea: In the same row, if columns: Payment Method and Location Column are any of the following (No value, 0, nan), AND if in the cols_to_check, in one of those columns in the SAME row still, if any one of them are also (no value, 0 , nan) then drop that entire row! 

it drops rows if:

Payment Method or Location are blank, and

There are 2 or more “bad” values in the important columns (Items, Quantity, PPU, Grand Total).

Item | Quantity | PPU | GT |     Location       | PM
0       NA         2      2       No value       No value -->drop (if >= 2 values are missing)
1      0          2       2    No value          No value -->dont drop 



In [36]:
cols_to_check = ["Items", "Quantity", "Price Per Unit", "Grand total"]

def is_blank(val):
    return pd.isna(val) or str(val).strip() == ""

df = df[~df.apply(
    lambda row: (
        # Check if either Payment Method or Location is blank
        is_blank(row["Payment Method"]) or is_blank(row["Location"])
    ) and (
        # Check if 2 or more of the important columns are bad
        sum(is_blank(row[col]) or str(row[col]).strip().lower() == "no value" or row[col] == 0 for col in cols_to_check) >= 2
    ),
    axis=1
)]


In [37]:
# to cehck
# Redefine is_blank for use here too
def is_blank(val):
    return pd.isna(val) or str(val).strip() == ""

# Check for rows that should have been dropped — sanity check
bad_rows = df[df.apply(
    lambda row: (
        is_blank(row["Payment Method"]) or is_blank(row["Location"])
    ) and (
        sum(is_blank(row[col]) or str(row[col]).strip().lower() == "no value" or row[col] == 0 for col in cols_to_check) >= 2
    ),
    axis=1
)]

# Print how many got through (should be 0)
print("Rows that still match the bad condition:", len(bad_rows))


Rows that still match the bad condition: 0


In [38]:
df.to_excel("cafe_data.xlsx", index=False)
